In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../Output_data/cities.csv")
weather_df

,Cities,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Margate,75,GB,1584313155,100,51.38,1.39,48.20,2.24
1,Bluff,87,NZ,1584313155,80,-46.60,168.33,55.99,10.00
2,Kapaa,75,US,1584312917,69,22.08,-159.32,80.60,11.41
3,Vila Franca do Campo,20,PT,1584313155,81,37.72,-25.43,53.60,4.70
4,Padang,84,ID,1584313155,88,-0.95,100.35,70.21,5.32
...,...,...,...,...,...,...,...,...,...
553,Mânăstirea,19,RO,1584313040,15,44.22,26.90,36.00,3.00
554,Bud,90,US,1584313040,83,39.45,-86.18,45.00,5.82
555,Magdalena,40,PH,1584313041,74,14.20,121.43,80.60,5.82
556,Olafsvik,100,IS,1584312857,78,64.89,-23.71,31.71,22.41


### Humidity Heatmap

Configure gmaps.
Use the Lat and Lng as locations and Humidity as the weight.
Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

In [5]:
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '450px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weather_df["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.
Drop any rows will null values.

In [23]:
narrowed_city_df = weather_df.loc[weather_df["Max Temp"] <= 80, :]
narrowed_city_df = narrowed_city_df.loc[narrowed_city_df["Max Temp"] >= 50, :]
narrowed_city_df = narrowed_city_df.loc[narrowed_city_df["Cloudiness"] == 0, :]
narrowed_city_df = narrowed_city_df.loc[narrowed_city_df["Wind Speed"] <=15, :]
narrowed_city_df = narrowed_city_df.loc[narrowed_city_df["Humidity"] <= 50, :]
narrowed_city_df = narrowed_city_df.loc[narrowed_city_df["Humidity"] >= 20, :]
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df

,Cities,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
316,Broken Hill,0,AU,1584312986,42,-31.95,141.43,64.40,11.41
365,Bahía Blanca,0,AR,1584312785,34,-38.72,-62.27,63.18,14.07
391,Florence,0,IT,1584312727,45,43.77,11.25,50.00,6.93
409,Riyadh,0,SA,1584312990,37,24.69,46.72,68.00,9.17
471,Forbes,0,AU,1584312847,47,-33.38,148.02,72.00,3.00
544,Aswān,0,EG,1584312844,43,24.09,32.91,69.80,8.05
550,Najrān,0,SA,1584313039,35,17.49,44.13,71.60,4.70


### Hotel Map

Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [28]:
hotel_df = narrowed_city_df.loc[:, ["Cities", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.rename(columns={"Cities": "City"})
hotel_df

,City,Country,Lat,Lng,Hotel Name
316,Broken Hill,AU,-31.95,141.43,
365,Bahía Blanca,AR,-38.72,-62.27,
391,Florence,IT,43.77,11.25,
409,Riyadh,SA,24.69,46.72,
471,Forbes,AU,-33.38,148.02,
544,Aswān,EG,24.09,32.91,
550,Najrān,SA,17.49,44.13,


In [29]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

results = []

for index, row in hotel_df.iterrows():

    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}")
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
    except (IndexError):
        print("Missing field/result...skipping")

Retrieving Results for Index 316
Retrieving Results for Index 365
Retrieving Results for Index 391
Retrieving Results for Index 409
Retrieving Results for Index 471
Retrieving Results for Index 544
Retrieving Results for Index 550
Missing field/result...skipping


In [30]:
hotel_df = hotel_df.drop([550])
hotel_df

,City,Country,Lat,Lng,Hotel Name
316,Broken Hill,AU,-31.95,141.43,Royal Exchange Hotel
365,Bahía Blanca,AR,-38.72,-62.27,Hotel Land Plaza
391,Florence,IT,43.77,11.25,Hotel Ginori al Duomo
409,Riyadh,SA,24.69,46.72,Hilton Garden Inn Riyadh Olaya
471,Forbes,AU,-33.38,148.02,Vandenberg Hotel
544,Aswān,EG,24.09,32.91,Sofitel Legend Old Cataract Aswan


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
hotel_locs = gmaps.symbol_layer(
    locations, 
    info_box_content=hotel_info
)

# Display Map
fig.add_layer(hotel_locs)
fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…